# 1. Introduction

<img src="intro1.png" alt="score" width="500" height="200" 
style="float:left;">

So the key technology for handling datasets that are this big is called big data. Unfortunately, these days, big data is a buzzword and everyone's got their own little definition. **I like to think of big data technology as any technology that requires distributed computing.** 

Case 1: if you have a database, but that database uses what's called sharding, that means user IDs, 1 to 1000 will be on one machine, whereas user IDs, 1001 to 2000 will be on another machine and so on.

Case 2: You can even sharding the files themselves. So if you have a humongous log file of all the events that are happening on your website and say the file is one terabyte large, then you might store pieces of that file across different machines around the world.

Case 3: Finally, you can distribute compute itself. So imagine you have to do a very long for loop, just like we have to do in matrix factorization. So we have to write for i in range N but N is 1.8 billion. So that's not going to happen on a single machine in any reasonable amount of time. Instead, the first 1000 users will be processed on one machine. The second 1000 users will be processed on another machine and so forth. This is what we're mostly interested in, at least for this course. Our actual data file is small enough to fit on our local machine, but the algorithm that we use to process it in particular matrix factorization, can be sped up by distributing the compute across multiple machines.

<img src="intro2.png" alt="score" width="500" height="200" style="float:left;">
<img src="intro3.png" alt="score" width="400" height="200" style="float:left;">

# Section Outline
<img src="intro4.png" alt="score" width="400" height="200" style="float:left;">

# 2. Spark install on local MacOS
Detailed steps are not included here.

After install, termal "spark-shell", test it with some code, and ":q" to exit.

What we'll actually be using is pyspark. It's still spark, but we can write our code in Python. This is the interface we'll be using for writing our matrix factorization code. So if you've gotten this far, then you're ready to head over to that lecture. In terminal, type "pyspark".

Basically, you need to install:
- Homebrew
- Xcode
- Java
- Scala
- apache-spark


# 3. MF in Spark on local machine
### meant to be pasted into console, not in jupyter notebook
Check the **spark.py**.

In [ ]:
# notes:
# you may have trouble with full dataset on just your local machine
# if you want to know what's in an RDD, use .take(n), ex:
# tmp = p.take(5)
# print(tmp)

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import os

# load in the data
data = sc.textFile("../large_files/movielens-20m-dataset/small_rating.csv")

# filter out header
header = data.first() #extract header
data = data.filter(lambda row: row != header)

# convert into a sequence of Rating objects
ratings = data.map(
  lambda l: l.split(',')
).map(
  lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))
)

# split into train and test
train, test = ratings.randomSplit([0.8, 0.2])

# train the model
K = 10
epochs = 10
model = ALS.train(train, K, epochs)

# evaluate the model

# train
x = train.map(lambda p: (p[0], p[1]))
p = model.predictAll(x).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = train.map(lambda r: ((r[0], r[1]), r[2])).join(p)
# joins on first item: (user_id, movie_id)
# each row of result is: ((user_id, movie_id), (rating, prediction))
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("train mse: %s" % mse)


# test
x = test.map(lambda p: (p[0], p[1]))
p = model.predictAll(x).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = test.map(lambda r: ((r[0], r[1]), r[2])).join(p)
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("test mse: %s" % mse)